In [927]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [928]:
matches1 = pd.read_csv("assets/2023-2024.csv")
matches2 = pd.read_csv("assets/2024-2025.csv")
matches = pd.concat([matches1, matches2])
matches

,Team,Date Time (US Eastern),Fixture,Home Team,Home Goal,Away Goal,Away Team,homeAway,foulsCommitted,yellowCards,...,longballPct,blockedShots,effectiveTackles,totalTackles,tacklePct,interceptions,effectiveClearance,totalClearance,updateTime,Unnamed: 37
0,Burnley,8/11/23 15:00,Manchester City at Burnley,Burnley,0.0,3.0,Manchester City,home,11.0,0.0,...,1.0,2.0,8.0,12.0,1.0,7.0,16.0,16.0,5/20/24 16:45,NaN
1,Manchester City,8/11/23 15:00,Manchester City at Burnley,Burnley,0.0,3.0,Manchester City,away,8.0,0.0,...,1.0,5.0,12.0,17.0,1.0,4.0,6.0,6.0,5/20/24 16:45,NaN
2,Arsenal,8/12/23 08:00,Nottingham Forest at Arsenal,Arsenal,2.0,1.0,Nottingham Forest,home,12.0,2.0,...,1.0,5.0,15.0,19.0,1.0,7.0,9.0,9.0,5/20/24 16:45,NaN
3,Nottingham Forest,8/12/23 08:00,Nottingham Forest at Arsenal,Arsenal,2.0,1.0,Nottingham Forest,away,12.0,2.0,...,0.0,1.0,10.0,17.0,1.0,8.0,16.0,16.0,5/20/24 16:45,NaN
4,AFC Bournemouth,8/12/23 10:00,West Ham United at AFC Bournemouth,AFC Bournemouth,1.0,1.0,West Ham United,home,9.0,1.0,...,0.0,3.0,6.0,17.0,0.0,7.0,10.0,10.0,5/20/24 16:45,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [929]:
matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])

""" matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])
matches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) """

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1072/1605438902.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])


' matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])\nmatches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) '

In [930]:
#advantages and disadvantages to consider when teams play against each other

matches["homeAway_code"] = matches["homeAway"].astype("category").cat.codes

matches["opponent_code"] = np.where(
    matches["Team"] == matches["Home Team"],
    matches["Away Team"].astype("category").cat.codes,
    matches["Home Team"].astype("category").cat.codes
)
#matches["hour"] = matches["Date Time (US Eastern)"].dt.hour.astype("Int64")
matches["day_code"] = matches["Date Time (US Eastern)"].dt.dayofweek
matches["date"] = matches["Date Time (US Eastern)"].dt.date

In [931]:
#determine the match result
matches["target"] = np.where(
    matches["Home Goal"] == matches["Away Goal"], 2,  
    np.where(
        (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] == matches["Home Team"]), 1,  
        np.where(
            (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] != matches["Home Team"]), 0,  
            np.where(
                (matches["Home Goal"] < matches["Away Goal"]) & (matches["Team"] == matches["Away Team"]), 1,  
                0  
            )
        )
    )
)

In [932]:
#began using machine learning 
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [933]:
#predictors = ["homeAway_code", "opponent_code", "hour", "day_code"]
predictors = [   "homeAway_code", "opponent_code", "day_code", 
        "shotsOnTarget", "accuratePasses", "accurateCrosses", 
        "accurateLongBalls", "blockedShots", "effectiveTackles", "effectiveClearance"
    ]

In [934]:
train = matches[matches["Date Time (US Eastern)"] <= '2024-05-01']

In [935]:
test = matches[matches["Date Time (US Eastern)"] > '2024-05-01']

In [936]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [937]:
preds = rf.predict(test[predictors])

In [938]:
accuracy = accuracy_score(test["target"], preds)
accuracy

0.5486111111111112

In [939]:
combine = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [940]:
pd.crosstab(index=combine["actual"], columns=combine["prediction"])

prediction,0,1,2
actual,,,
0,79,28,3
1,29,75,6
2,37,27,4


In [941]:
precision_score(test["target"], preds, average='macro', zero_division=0)

0.4764809902740937

In [942]:
grouped_matches = matches.groupby("Team")
""" group = grouped_matches.get_group("West Ham United")
group """

' group = grouped_matches.get_group("West Ham United")\ngroup '

In [943]:
""" 
This function uses the stats from the previous 3 games 
to determine the team's condition and predict the next game. 
"""
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date Time (US Eastern)")
    rolling_stats = group[cols].shift(1).rolling(window=3).mean()
    for col, new_col in zip(cols, new_cols):
            group[new_col] = rolling_stats[col]
    group = group.dropna(subset=new_cols)
    return group

In [944]:
columns = [
    "totalShots", "possessionPct", 
    "totalPasses", "totalTackles",
    ]
new_columns = [f"{c}_rolling" for c in columns]

In [945]:
""" rolling_averages(group, columns, new_columns) """
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))
matches_rolling.droplevel("Team")
matches_rolling.index = range(matches_rolling.shape[0])
#matches_rolling

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1072/2647377087.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))


In [946]:
def make_predictions(data, predictors):
    train = data[data["Date Time (US Eastern)"] <= '2024-05-01']
    test = data[data["Date Time (US Eastern)"] > '2024-05-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combine = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds, average='macro', zero_division=0)
    return combine, precision

In [947]:
combined, precision = make_predictions(matches_rolling, predictors + new_columns)

combined = combined.merge(matches_rolling[["date", "Team" ,"Home Team", "Away Team", "target"]], left_index=True, right_index=True)

combined_liverpool = combined[combined["Team"] == "Manchester United"]
combined_liverpool


,actual,prediction,date,Team,Home Team,Away Team,target
623,0,0,2024-05-06,Manchester United,Crystal Palace,Manchester United,0
624,0,0,2024-05-12,Manchester United,Manchester United,Arsenal,0
625,1,1,2024-05-15,Manchester United,Manchester United,Newcastle United,1
626,1,0,2024-05-19,Manchester United,Brighton & Hove Albion,Manchester United,1
627,1,2,2024-08-16,Manchester United,Manchester United,Fulham,1
628,0,0,2024-08-24,Manchester United,Brighton & Hove Albion,Manchester United,0
629,0,0,2024-09-01,Manchester United,Manchester United,Liverpool,0
630,1,1,2024-09-14,Manchester United,Southampton,Manchester United,1
631,2,1,2024-09-21,Manchester United,Crystal Palace,Manchester United,2
632,0,0,2024-09-29,Manchester United,Manchester United,Tottenham Hotspur,0


In [948]:
precision

0.4361890166028097

In [949]:
accuracy

0.5486111111111112